In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.model_selection import KFold
from sklearn import metrics
import seaborn as sns 
from numpy import *
from sklearn.model_selection import GridSearchCV

In [16]:
def artificial_cv(mod,total_fold,x,y):
    train_index = []
    test_index = []
    r2 = []
    kf = KFold(n_splits = total_fold,shuffle= True,random_state = 0)
#     kf = KFold(n_splits = total_fold)
    for train_i, test_i in kf.split(x):
        train_index.append(train_i)
        test_index.append(test_i)
    x.index = range(0, len(x)) 
    y.index = range(0, len(y))
    for i in range(0,total_fold):
        train_x = x.loc[train_index[i]]
        train_y = y.loc[train_index[i]]
        test_x = x.loc[test_index[i]]
        test_y = y.loc[test_index[i]]
        mod.fit(train_x,train_y.values.ravel())
        y_pred = mod.predict(test_x)
        r2_i = metrics.r2_score(test_y, y_pred)
        r2.append(r2_i)
    cv = np.mean(r2)
    return cv

In [3]:
def Read_GTiff(filename):
    ds = gdal.Open(filename)
    data = ds.ReadAsArray()
    return data
def Get_LatLon_GTiff(fname):
    """
    Return two numpy arrays of lat,lon of the tif file.
    """
    ds = gdal.Open(fname)
    size_x = ds.RasterXSize
    size_y = ds.RasterYSize
    originX, pixelWidth, _, originY, _, pixelHeight = ds.GetGeoTransform()
    lon = np.arange(originX,originX+pixelWidth*size_x,pixelWidth)
    lat = np.arange(originY,originY+pixelHeight*size_y,pixelHeight)
    return lat,lon

def Read_Gtiff_data_latlon(fname):
    """
    Return lat,lon,data
    """
    lat,lon = Get_LatLon_GTiff(fname)
    data = Read_GTiff(fname)
    return lat,lon,data

In [4]:
from osgeo import gdal,ogr,osr
def array2raster(newRasterfn,rasterOrigin,pixelWidth,pixelHeight,array):

    cols = array.shape[1]
    rows = array.shape[0]
    originX = rasterOrigin[0]
    originY = rasterOrigin[1]

    driver = gdal.GetDriverByName('GTiff')
    outRaster = driver.Create(newRasterfn, cols, rows, 1, gdal.GDT_Float32)
    outRaster.SetGeoTransform((originX, pixelWidth,0 ,originY, 0 , pixelHeight))
    outband = outRaster.GetRasterBand(1)

    outband.WriteArray(array)
    outRasterSRS = osr.SpatialReference()
    outRasterSRS.ImportFromEPSG(4326)
    outRaster.SetProjection(outRasterSRS.ExportToWkt())
    outband.FlushCache()

In [5]:
def upscale(x,rfc):
    col_names = []
    n = x.shape[1]*x.shape[2]
    df_pred = pd.DataFrame(index=np.arange(n))
    df_x = pd.DataFrame(index=np.arange(n))
    
    for i in range(x.shape[0]):
        x_ = x[i].flatten()
        df_x['variable_{}'.format(i)] = x_
    y = rfc.predict(df_x)
    df_pred['result'] = y
#     df_pred.loc[nan_list,'result']= np.nan
    y = df_pred.values
    y = y.reshape((x.shape[1],x.shape[2])) 
    return y

In [6]:
dataset = pd.read_excel("F:/site_biomass/1.China/train_221014.xlsx")

In [7]:
var =  ["lon","lat","AK","altitude","AN","AP","aspect",
        "awc","AWCtS",
        "BDRLOG","BDTICM","BLDFIE","CECSOL",
        "CLYPPT","DIF","LOS","OCDENS","OCSTHA",
        "ORCDRC","PHIKCL","PHIHOX","P_pet","P_pre","P_tmp",
        "slope","SLTPPT",
        "SNDPPT","WWP"]
result = ["rate"]

In [9]:
dataset_age40=dataset_age40.query('`rate`<16.425881834356233')

# needle

In [10]:
needle=dataset_age40.query('`N_B` == 1')

In [11]:
X_n = needle[var]
Y_n = needle[result]

In [12]:
rfc_n = RandomForestRegressor(n_estimators=120, max_features=4,
                            max_depth=14,bootstrap=True,oob_score=True,
                            n_jobs=-1)

In [13]:
 rfc_n.fit(X_n, Y_n.values.ravel())

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=14, max_features=4, max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=120, n_jobs=-1, oob_score=True,
                      random_state=None, verbose=0, warm_start=False)

In [ ]:
artificial_cv(rfc_n,3,X_n,Y_n)

In [ ]:
rfc.oob_score_

In [18]:
from sklearn.inspection import plot_partial_dependence

In [ ]:
import os
import glob
a = []
inws = r"D:/site_biomass/9_28_11/"
values = glob.glob(os.path.join(inws, "*.tif"))
for i,val in zip(range(0,28),values):
    print("{}  ".format(i),val)
    var = Read_GTiff(val)
    lat,lon = G et_LatLon_GTiff(val)
    lonmx = np.argwhere(lon<=135)
    lonmn = np.argwhere(lon>=73)
    latmx = np.argwhere(lat<=54)
    latmn = np.argwhere(lat>=18.5)
    var = var[latmx.min():latmn.max(),lonmn.min():lonmx.max()]
    a.append(var)
a = np.array(a) 

In [54]:
modeltif = Read_GTiff("D:/site_biomass/9_28_11/AK.tif")
lat,lon = Get_LatLon_GTiff("D:/site_biomass/9_28_11/AK.tif")
lonmx = np.argwhere(lon<=135)
lonmn = np.argwhere(lon>=73)
latmx = np.argwhere(lat<=54)
latmn = np.argwhere(lat>=18.5)
print(latmx.min(),latmn.max(),lonmn.min(),lonmx.max())

0 3549 0 6199


In [38]:
lat_var = np.full([3550, 6200], np.nan)

In [39]:
lat_var = np.tile(lat[0:3550],(6200,1))

In [40]:
lat_var = lat_var.transpose()

In [41]:
lat_var.shape

(3550, 6200)

In [42]:
lat,lon = Get_LatLon_GTiff("D:/site_biomass/9_28_11/AK.tif")
pixelWidth = lon[1]-lon[0]
pixelHeight = lat[1]-lat[0]
rasterOrigin = (lon[0],lat[0])
array = lat_var
newRasterfn = "D:/site_biomass/9_28_11/2_lat.tif"
array2raster(newRasterfn,rasterOrigin,pixelWidth,pixelHeight,array)

In [43]:
lon_var = np.full([3550, 6200], np.nan)

In [44]:
lon_var = np.tile(lon[0:6200],(3550,1))

In [45]:
array = lon_var
newRasterfn = "D:/site_biomass/9_28_11/1_lon.tif"
array2raster(newRasterfn,rasterOrigin,pixelWidth,pixelHeight,array)

In [48]:
y_N = upscale(a,rfc)

In [51]:
array = y_N
newRasterfn = "D:/site_biomass/upscale/dif_n.tif"
array2raster(newRasterfn,rasterOrigin,pixelWidth,pixelHeight,array)

# BROAD

In [20]:
broad=dataset_age40.query('`N_B` == 2')

In [21]:
X_b = broad[var]
Y_b = broad[result]

In [17]:
cv = []
oob = []
from sklearn import metrics
for i in range(100,400,10):
    rfc = RandomForestRegressor(
                                n_estimators=i+1,
                                bootstrap=True,
                                oob_score=True,
                                n_jobs=-1,
    )
    score_cv = artificial_cv(3,X_b,Y_b)
    cv.append(score_cv)
#     cv.append(-score_cv)
    rfc.fit(X_b, Y_b.values.ravel())
    oob.append(rfc.oob_score_)

In [ ]:
cv_max = np.max(cv)
print('cv：{}'.format(cv_max),
      '{}'.format(cv.index(cv_max)*10+101))
oob_max = np.max(oob)
print('oob：{}'.format(oob_max),
      '{}'.format(oob.index(oob_max)*10+101))

x = np.arange(101,401,10)
# plt.subplot(111)
plt.figure(figsize=(14, 5))
plt.xticks(x)

plt.plot(x, cv, label='cv')
plt.plot(x, oob,label='oob' )

plt.legend(prop={'family' : 'Times New Roman', 'size': 12})

plt.show()

In [ ]:
cv = []
oob = []

from sklearn import metrics
for i in range(0,25,1):
    rfc = RandomForestRegressor(
                                n_estimators=371,
                                max_depth = i+1,
                                bootstrap=True,
                                oob_score=True,
                                n_jobs=-1
    )
    score_cv = artificial_cv(3,X_b,Y_b)
    cv.append(score_cv)
#     cv.append(-score_cv)
    rfc.fit(X_b, Y_b.values.ravel())
    oob.append(rfc.oob_score_)

In [ ]:
cv_max = max(cv)
print('cv：{}'.format(cv_max),
      '{}'.format(cv.index(cv_max)*1+1))
oob_max = max(oob)
print('oob：{}'.format(oob_max),
      '{}'.format(oob.index(oob_max)*1+1))

x = np.arange(1,26,1)
# plt.subplot(111)
plt.figure(figsize=(14, 5))
plt.xticks(x)
plt.plot(x, cv, label='cv')
plt.plot(x, oob,label='oob' )

plt.legend(prop={'family' : 'Times New Roman', 'size': 12})

plt.show()

In [30]:
cv = []
oob = []
from sklearn import metrics
for i in range(2,28):
    rfc = RandomForestRegressor(
                                n_estimators=120,
                                max_features=i,
                                max_depth=21,
                                bootstrap=True,
                                oob_score=True,
                                n_jobs=-1,
                                random_state = 0
    )
    score_cv = artificial_cv(3,X_b,Y_b)
    cv.append(score_cv)
#     cv.append(-score_cv)
    rfc.fit(X_b, Y_b.values.ravel())
    oob.append(rfc.oob_score_)

In [ ]:
cv_max = np.max(cv)
print('cv：{}'.format(cv_max),
      '{}'.format(cv.index(cv_max)+1))
oob_max = np.max(oob)
print('oob：{}'.format(oob_max),
      '{}'.format(oob.index(oob_max)+1))


x = np.arange(2,28)
# plt.subplot(111)
plt.figure(figsize=(14, 5))
plt.xticks(x)

plt.plot(x, cv, label='cv')
plt.plot(x, oob,label='oob' )


plt.legend(prop={'family' : 'Times New Roman', 'size': 12})

plt.show()

In [23]:
rfc_b = RandomForestRegressor(n_estimators=371, max_features=7,
                            max_depth=21,bootstrap=True,oob_score=True,
                            n_jobs=-1,random_state=0)

In [24]:
artificial_cv(rfc_b,3,X_b,Y_b)

0.6047872093379661

In [26]:
rfc_b.fit(X_b,Y_b.values.ravel())

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=21, max_features=7, max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=371, n_jobs=-1, oob_score=True,
                      random_state=0, verbose=0, warm_start=False)

In [ ]:
print('特征排序')

feature_names = list(X_b.columns)

feature_importances = rfc.feature_importances_
indics = np.argsort(feature_importances)[::-1]

for index in indics:
    print(' %s (%f)' %(feature_names[index], feature_importances[index]))

In [66]:
y_B = upscale(a,rfc)

In [67]:
array = y_B
newRasterfn = "D:/site_biomass/upscale/dif_b.tif"
array2raster(newRasterfn,rasterOrigin,pixelWidth,pixelHeight,array)

# mixed

In [27]:
mixed=dataset_age40.query('`N_B` == 3')

In [28]:
X_m = mixed[var]
Y_m = mixed[result]

In [44]:
cv = []
oob = []
from sklearn import metrics
for i in range(100,400,20):
    rfc = RandomForestRegressor(
                                n_estimators=i+1,
                                bootstrap=True,
                                oob_score=True,
                                n_jobs=-1,

    )
    score_cv = artificial_cv(3,X_m,Y_m)
    cv.append(score_cv)
#     cv.append(-score_cv)
    rfc.fit(X_m, Y_m.values.ravel())
    oob.append(rfc.oob_score_)

In [ ]:
cv_max = np.max(cv)
print('cv：{}'.format(cv_max),
      '{}'.format(cv.index(cv_max)*20+101))
oob_max = np.max(oob)
print('oob：{}'.format(oob_max),
      '{}'.format(oob.index(oob_max)*20+101))

x = np.arange(101,401,20)
# plt.subplot(111)
plt.figure(figsize=(14, 5))
plt.xticks(x)

plt.plot(x, cv, label='cv')
plt.plot(x, oob,label='oob' )

plt.legend(prop={'family' : 'Times New Roman', 'size': 12})

plt.show()

In [48]:
cv = []
oob = []

from sklearn import metrics
for i in range(0,25,1):
    rfc = RandomForestRegressor(
                                n_estimators=120,
                                max_depth = i+1,
                                bootstrap=True,
                                oob_score=True,
                                n_jobs=-1
    )
    score_cv = artificial_cv(3,X_m,Y_m)
    cv.append(score_cv)
#     cv.append(-score_cv)
    rfc.fit(X_m,Y_m.values.ravel())
    oob.append(rfc.oob_score_)

In [ ]:
cv_max = max(cv)
print('cv：{}'.format(cv_max),
      '{}'.format(cv.index(cv_max)*1+1))
oob_max = max(oob)
print('oob：{}'.format(oob_max),
      '{}'.format(oob.index(oob_max)*1+1))

x = np.arange(1,26,1)
# plt.subplot(111)
plt.figure(figsize=(14, 5))
plt.xticks(x)
plt.plot(x, cv, label='cv')
plt.plot(x, oob,label='oob' )

plt.legend(prop={'family' : 'Times New Roman', 'size': 12})

plt.show()

In [50]:
cv = []
oob = []
from sklearn import metrics
for i in range(2,28):
    rfc = RandomForestRegressor(
                                n_estimators=120,
                                max_features=i,
                                max_depth=3,
                                bootstrap=True,
                                oob_score=True,
                                n_jobs=-1,
                                random_state = 0
    )
    score_cv = artificial_cv(3,X_m,Y_m)
    cv.append(score_cv)
#     cv.append(-score_cv)
    rfc.fit(X_m, Y_m.values.ravel())
    oob.append(rfc.oob_score_)

In [ ]:
cv_max = np.max(cv)
print('cv：{}'.format(cv_max),
      '{}'.format(cv.index(cv_max)+1))
oob_max = np.max(oob)
print('oob：{}'.format(oob_max),
      '{}'.format(oob.index(oob_max)+1))


x = np.arange(2,28)
# plt.subplot(111)
plt.figure(figsize=(14, 5))
plt.xticks(x)

plt.plot(x, cv, label='cv')
plt.plot(x, oob,label='oob' )


plt.legend(prop={'family' : 'Times New Roman', 'size': 12})

plt.show()

In [30]:
rfc_m = RandomForestRegressor(n_estimators=120, max_features=7,
                            max_depth=6,bootstrap=True,oob_score=True,
                            n_jobs=-1,random_state=0)

In [ ]:
artificial_cv(rfc_m,3,X_m,Y_m)

In [32]:
rfc_m.fit(X_m,Y_m.values.ravel())

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=6, max_features=7, max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=120, n_jobs=-1, oob_score=True,
                      random_state=0, verbose=0, warm_start=False)

In [83]:
y_M = upscale(a,rfc)

In [84]:
array = y_M
newRasterfn = "D:/site_biomass/upscale/dif_m.tif"
array2raster(newRasterfn,rasterOrigin,pixelWidth,pixelHeight,array)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 6))
tree_disp.plot(ax=[ax1, ax2], line_kw={"label": "Decision Tree"})
mlp_disp.plot(ax=[ax1, ax2], line_kw={"label": "Multi-layer Perceptron",
                                      "c": "red"})
ax1.legend()
ax2.legend()